<a href="https://colab.research.google.com/github/TheReallyBigApple/CunyAssignments/blob/main/Data604/SimpyProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#                       CUNY MSDS Data604 

#                            Tom Buonora



# Final Project using Simpy

In [29]:
try:
    import simpy
except ImportError:
    !pip install simpy
    import simpy

import random
import numpy as np
import pandas as pd
from enum import Enum
from datetime import datetime

In [30]:
if __name__ == "__main__":
    main()

Starting  07/16/2021, 15:31:12
 
 


,Servers,PacketSize,Submitted,Processed,Pct_Processed
0,1.0,1.0,4513.0,2674.0,0.59
1,1.0,20.0,8619.0,6739.0,0.78
2,1.0,30.0,8805.0,6931.0,0.79
3,1.0,40.0,7627.0,5739.0,0.75
4,1.0,50.0,8249.0,4150.0,0.50
5,3.0,1.0,8603.0,6706.0,0.78
6,3.0,20.0,7032.0,5190.0,0.74
7,3.0,30.0,5238.0,3403.0,0.65
8,3.0,40.0,12715.0,10844.0,0.85
9,3.0,50.0,12431.0,10117.0,0.81




The best performance was with  3.0  servers and  40.0 packets.


Ending  07/16/2021, 15:35:16


**This simulation takes about 5 minutes. Output will be above**






**To control logging, set the following attributes in the Debug class**
- g_logging_on   - True to enable additional logging

- g_debug_level  - Info, Warn or Verbose

- g_path         - the directory you wish to log to

In [31]:
class Debug():
    Info = 1
    Warn = 2
    Verbose = 3

    g_logging_on = False
    g_debug_level=Verbose
    g_path= r'C:\Users\arono\CUNY\Data604_Project\\'
        
    log_file=g_path + "simlog.txt"

    def open_log():
        if (Debug.g_logging_on):
            Debug.g_debug_file=open(Debug.log_file,"w")
    
    def close_log():
        if (Debug.g_logging_on):
            Debug.g_debug_file.close()
        
        
    def log(debug_level,str):
        if (Debug.g_logging_on):
            if (Debug.g_debug_level >= debug_level):
                Debug.g_debug_file.write(str + "\n")
            
            
            
    def post_csv(df, filename):
        if (Debug.g_logging_on):
            file=Debug.g_path + filename
            df.to_csv(file)
            
            

# Event Server class

In [32]:
class EventServer(object):
    def __init__(self, env, num_servers, packet_size):
        self.env = env
 
        self.num_servers=num_servers
        self.packet_size=packet_size

         
        self.stats = pd.DataFrame(columns=["EType", "Events", "BaseMS", "MSFactor", "ProcessTime", "Now", "PE_Start", "PE_End"])
    
        self.g_pe_start=0          # the # of times process event is called with X # of events
        self.g_pe_end=0 
        self.g_tot_events_processed=0
        self.g_tot_events_submitted=0
        self.g_tot_time=0

        self.worker = simpy.Resource(env, self.num_servers)
       
        
            # The baseline performance is a stochastic sampling
            # from a logistic distribution ( like a normal one except with longer tails )
            #   absolute values, mean=1, std.dev=1, so it approximates the actual range of base line performance in milliseconds per event
        self.base_ms_dist = (abs(np.random.logistic(loc=1, scale=1, size=(200))))
    
   
    def process_event(self, ms, pe_id):   # simulate the processing time of the tran server
        
        start = self.env.now         # env.now is an integer
        
        yield self.env.timeout(ms)
        self.g_tot_events_processed+=1
        self.g_tot_time+=ms

        end= self.env.now
        
        Debug.log(Debug.Verbose, "           process_event :  start {0}  end {1}  pe_id = {2}  sub/proc {3} {4}".format( \
                  round(start,3), round(end,3),   pe_id,  self.g_tot_events_submitted,  self.g_tot_events_processed))
   

# Event Type 1

In [33]:

class EventType1(object):
    def __init__(self,eventserver):
        self.name = "EventType 1  "
         
        self.g_event_total=0
        
        self.performance_factors=np.zeros(eventserver.packet_size)
        
        self.create_performance_factors(eventserver)
    
        Debug.log(Debug.Warn, "\n {0} Num. Server Factor : {1}".format(self.name,self.svr_factor))
        Debug.log(Debug.Warn, "{0} Performance Factors : {1} \n".format(self.name, self.performance_factors))

   
        
    
    # return the distribution of events expected for a given time unit i.e. 20 milliseconds
    def events_per_time_unit_dist(self):
        dist = np.append(np.repeat(0,344), np.repeat(1,145))
        dist = np.append(dist, np.repeat(2,262))
        dist = np.append(dist,  np.repeat(3,213))
        dist = np.append(dist,  np.repeat(5,85))
        dist = np.append(dist,  np.repeat(7,35))
        dist = np.append(dist,  np.repeat(15,2))
        dist = np.append(dist,  np.repeat(35,2))
        dist = np.append(dist,  np.repeat(200,1))
        dist = np.append(dist,  np.repeat(500,1))
        dist = np.append(dist,  np.repeat(1000,1))
        
        return dist

      
     
    def create_performance_factors(self, eventserver):
         '''
         
         Parameters
         ----------
         packet_size : the system setting to how many events can we process in bulk
         server_instances : the number of servers running simultaneously

         Returns 
         -------
         creates an array of factors. Each factor is multiplied by the baseline processing time
             to get the true processing time of the event

         '''
             # optimal # of servers for event 1 is 3
             # note this factor ignores the improvement of concurrent processing since that occurs by simpy resources
             #   the factor is the penalty of resource conention and memory usage
             
         self.e_svr_arr = np.array([1.7, 1.3, 1.05, .90, .85,.75 ,.82, 1.13, 1.4, 1.4,1.8,2.5,3,3.2])
         self.svr_factor=self.e_svr_arr[eventserver.num_servers]
            
        
        
         # create the packet performance array
            # the packet processor has ways of processing in bulk what the transaction servers would do iteratively
              # optimal packet size for event 1 is 50 which is the maximum allowed

         e1=np.array([2,1.8,1.7,1.6,1.5,1.4,1.3,1.2,1.1,1.05])
         e2=np.concatenate((e1,e1,e1,e1,e1))
         e3=np.sort(e2)[::-1]          # sort descending so its fairly linear from 2 to 1
         self.e_pkt_arr=e3
  
         
         # for every possible packet size populate a factor for the packet and for 
             #  the overall performance
                  # i.e. if only 4 events are being processed, 
                  # then we use the factor for a packet size of 4
         for i in range(eventserver.packet_size):
                self.performance_factors[i] = self.svr_factor * self.e_pkt_arr[i]


# Event Type 2

In [34]:

         
class EventType2(object):
    def __init__(self,eventserver):
        self.name = "EventType 2  "
         
        self.g_event_total=0
        
        self.performance_factors=np.zeros(eventserver.packet_size)
        self.create_performance_factors(eventserver)
        
        Debug.log(Debug.Warn, "\n {0} Num. Server Factor : {1}".format(self.name,self.svr_factor))
        Debug.log(Debug.Warn, "{0} Performance Factors : {1} \n".format(self.name, self.performance_factors))

       
        
       
    
    def events_per_time_unit_dist(self):
        dist = np.append(np.repeat(0,394), np.repeat(1,132))
        dist = np.append(dist, np.repeat(2,101))
        dist = np.append(dist,  np.repeat(3,60))
        dist = np.append(dist,  np.repeat(5,17))
        dist = np.append(dist,  np.repeat(7,9))
        dist = np.append(dist,  np.repeat(15,3))
        dist = np.append(dist,  np.repeat(20,2))
       
        return dist

      
     
    def create_performance_factors(self, eventserver):
       
                   # optimal servers for event 2 is 7
        self.e_svr_arr = np.array([3.4, 2.6, 1.9, 1.4, 1.2, 1.1, 1, 1.1, 1.2, 2.1, 2.5, 3.4, 3.8, 4.4, 5.2, 6 ])
        self.svr_factor = self.e_svr_arr[eventserver.num_servers-1]
 
      
        e1=np.array([1.7,1.3,1.2,1.1,1.05])
        e2=e1*-1
        e3=np.concatenate((e1,e1,e1,e1,e2,e2,e2,e2,[1.9,2,2,2.3,2.7,4.1,5,5,5,5]))
        e4=np.sort(e3)
        e5=abs(e4)
        self.e_pkt_arr=e5
     
        for i in range(eventserver.packet_size):
             self.performance_factors[i] = self.svr_factor * self.e_pkt_arr[i]

    
    
    

# Event Type 3

In [35]:

         
class EventType3(object):
    def __init__(self,eventserver):
        self.name = "EventType 3  "
        self.g_event_total=0
      
        self.performance_factors=np.zeros(eventserver.packet_size)
        
        self.create_performance_factors(eventserver)
        
        Debug.log(Debug.Warn, "\n {0} Num. Server Factor : {1}".format(self.name,self.svr_factor))
        Debug.log(Debug.Warn, "{0} Performance Factors : {1} \n".format(self.name, self.performance_factors))

        
        
    
    def events_per_time_unit_dist(self):
        dist = np.append(np.repeat(0,633), np.repeat(1,45))
        dist = np.append(dist, np.repeat(2,27))
        dist = np.append(dist,  np.repeat(3,11))
        dist = np.append(dist,  np.repeat(5,2))
        dist = np.append(dist,  np.repeat(50,1))
        dist = np.append(dist,  np.repeat(1000,1))
 
        
        return dist

      
     
    def create_performance_factors(self, eventserver):
             
          # optimal server size for event 3 is 5
        self.e_svr_arr = np.array([3.9, 3.4, 3.2, 2.5, 2.1, 2.4, 3.0, 3.2, 4.4, 4.4, 4.7, 5.8, 6.4, 7.1, 7.6, 7.8 ])
        self.svr_factor = self.e_svr_arr[eventserver.num_servers-1]
  
                # the optimal # is 10 but degrades a lot over 30
        e1=np.array([1.7,1.3,1.2,1.1,1.05])
        e2=e1*-1
        e3=np.concatenate((e1,e1,e1,e1,e1,e1,e2,e2,[1.9,2.4,2.4,2.5,2.7,2.7,2.8,3,3,3]))
        e4=np.sort(e3)
        self.e_pkt_arr=abs(e4)

          
        for i in range(eventserver.packet_size):
             self.performance_factors[i] = self.svr_factor * self.e_pkt_arr[i]

      

# Event Type 4

In [36]:


class EventType4(object):
    def __init__(self, eventserver):
        self.name = "EventType 4  "
        self.g_event_total=0
        
  

                     # set up the factors and create a fit for this event
        self.packet_factors=np.zeros(eventserver.packet_size)

        self.performance_factors=np.zeros(eventserver.packet_size)
        
        self.create_performance_factors(eventserver)

        Debug.log(Debug.Warn, "\n {0} Num. Server Factor : {1}".format(self.name,self.svr_factor))
        Debug.log(Debug.Warn, "{0} Performance Factors : {1} \n".format(self.name, self.performance_factors))


    
    def events_per_time_unit_dist(self):
        dist = np.append(np.repeat(0,687), np.repeat(1,25))
        dist = np.append(dist, np.repeat(2,7))
        dist = np.append(dist,  np.repeat(5,1))
        
        return dist

      
     
    def create_performance_factors(self, eventserver):

              # optimal server size for event 4 is 2
        self.e_svr_arr = np.array([3.2, 3.05, 3.2, 3.4, 3.9, 4.6, 5.4, 5.6, 5.6, 5.6, 6, 6, 6.5, 6.6 ])
        self.svr_factor = self.e_svr_arr[eventserver.num_servers-1]
        
           # optimal packet size for event 4 is 1
        self.e_pkt_arr=np.array([1,1.3,1.4,1.5,1.25,1.55,2.1,3.2,4.4,5.8,6.4,8])
        e1=np.array([2,3.3,3.5,4.1,4.45,5,5,6,7,7])
        e3=np.concatenate(([1],e1,e1,e1,e1,[8,8,8,9,9,9,9,9,9]))
        e4=np.sort(e3)
        self.e_pkt_arr=e4

        for i in range(eventserver.packet_size):
            self.performance_factors[i] = self.svr_factor * self.e_pkt_arr[i]

     
     

# Mainline Control Functions

In [37]:
def process_events(env, events, ms, eventserver):
    
    eventserver.g_pe_start+=1    # keep track of how many
    
    # technically each server is processing an event one at a time
    pe_id="{0}_{1}_{2}".format(round(ms,4), events, eventserver.g_pe_start)
    
    Debug.log(Debug.Verbose, "     process_events entering {0}   pe_id = {1}".format(int(env.now), pe_id))

        
    for i in range(1,events):
         with eventserver.worker.request() as req:
             yield req
             yield env.process(eventserver.process_event(ms, pe_id))
             
    eventserver.g_pe_end+=1
    
    Debug.log(Debug.Verbose, "     process_events leaving {0}   pe_id = {1}".format(int(env.now), pe_id))
    

In [38]:

def event_type_submitter(env, eventserver, etype):
    
        # get the frequency distribution of this type of event AND THEN
        #    get the # of events to submit in this time cycle
    e_dist = etype.events_per_time_unit_dist()
    
    
    
    e_events = random.choice(e_dist)    

    if (e_events==0):
        return
    
    
            # record the # of events to our audit    
    eventserver.g_tot_events_submitted+=e_events
    etype.g_event_total+=e_events
    
    
        # if the number of events exceeds the packet limit, use the largest factor we have
    p1=e_events
    if (p1 > eventserver.packet_size):
        p1=eventserver.packet_size

        # get a random sample from our baseline millisecond distribution AND THEN
        #    calculate a processing time for this set of events
        
  
    
    base_ms = random.choice(eventserver.base_ms_dist)
    ms_factor = etype.performance_factors[p1-1]           # note the lowest packet size is 0


    process_time=base_ms * ms_factor
        
    if ( e_events > 0):
  
        env.process(process_events(env, e_events, process_time, eventserver))
  
  
        eventserver.stats=eventserver.stats.append({"EType": etype.name, "Events" : \
                            e_events, "BaseMS" : round(base_ms,3), "MSFactor" : round(ms_factor,3), \
                            "ProcessTime" : round(process_time,3), "Now" : env.now, \
                                "PE_Start" : eventserver.g_pe_start, "PE_End" : eventserver.g_pe_end}, \
                                                   ignore_index=True)



In [39]:
def event_server_controller(env, eventserver):
    

    e1 = EventType1(eventserver)
    e2 = EventType2(eventserver)
    e3 = EventType3(eventserver)
    e4 = EventType4(eventserver)    
 
    Debug.log(Debug.Info, "Running {0} servers and {1} packets".format(eventserver.num_servers,eventserver.packet_size))
    
    #  submit every 40 time units 
       # note : this is not meant to represent a sleep of 800 milliseconds
        # its meant to simulate a FIFO queue with an element of grouping trades for packet processing
        # the analysists have agreed to this
    
    while(1):
        yield env.timeout(50)
        event_type_submitter(env, eventserver, e1)
        event_type_submitter(env, eventserver, e2)
        event_type_submitter(env, eventserver, e3)
        event_type_submitter(env, eventserver, e4)
        

In [40]:
def run_simulation(run_window):

    summarystats = pd.DataFrame(columns=["Servers", "PacketSize", "Submitted", "Processed"])
      
            #  1 <= servers <= 12 
            #  1 <= packet size <= 50
    for servers in np.arange(1, 12, 2):      
        for packet_size in (1,20,30,40,50):
            
            env = simpy.Environment()
            eventserver = EventServer(env, servers, packet_size)
            
            env.process(event_server_controller(env, eventserver))
            env.run(until=run_window)     
    

            
            if (eventserver.g_tot_events_submitted > 0):
                summarystats=summarystats.append({ \
                       "Servers" : eventserver.num_servers, \
                       "PacketSize" : eventserver.packet_size , \
                       "Submitted" : eventserver.g_tot_events_submitted, \
                       "Processed" : eventserver.g_tot_events_processed, \
                       "Pct_Processed" : eventserver.g_tot_events_processed/eventserver.g_tot_events_submitted}, \
                            ignore_index=True)
                                                          
            stats_file="stats_svr_{0}_ps_{1}.csv".format(servers, packet_size)
            Debug.post_csv(eventserver.stats, stats_file)
            
            
    
            event_sum=eventserver.stats.groupby(["EType"])["Events"].sum()
            Debug.log(Debug.Warn,event_sum.to_string())
            
 
    print(" ")
    print(" ")
    pd.set_option('display.precision', 2)
    display(summarystats)


    Debug.post_csv(summarystats, "summary.csv")
 
    
    
    
    # isolate the best performing simulation
    best_perf = summarystats[summarystats['Pct_Processed']==summarystats.Pct_Processed.max()].index.tolist()
    maxi = best_perf[0]
    best_servers = summarystats.loc[maxi].Servers
    best_packets = summarystats.loc[maxi].PacketSize
    
    print("")
    print("")
    
    print("The best performance was with ", best_servers, " servers and ", best_packets, "packets.")

In [41]:

    
def main():
    
    ts = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
    print("Starting ", ts)
    Debug.open_log()
    
    random.seed(60)        # for the random.choice of the event volumes and the baseline distributin
    np.random.seed(60)     # for the np.random.logistic creation of the  baseline distribution 
    
    
    
            # 60 secs per minute 60 minutes per hour so 3600 secs per hour or 3,600,000 ms per hour
                       # each time unit is 20 milliseconds so well run for 72,000 Time Units
    four_hours=72000                   
    run_simulation(four_hours)

    
    Debug.close_log()
    
    print("")
    print("")
    
    ts = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
    print("Ending ", ts)

    
